In [10]:
# !pip install langchain chromadb  tiktoken pypdf langchain_groq langchain_community

In [2]:
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document

In [3]:
doc1 = Document(
        page_content="Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.",
        metadata={"team": "Royal Challengers Bangalore"}
    )
doc2 = Document(
        page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.",
        metadata={"team": "Mumbai Indians"}
    )
doc3 = Document(
        page_content="MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.",
        metadata={"team": "Chennai Super Kings"}
    )
doc4 = Document(
        page_content="Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.",
        metadata={"team": "Mumbai Indians"}
    )
doc5 = Document(
        page_content="Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.",
        metadata={"team": "Chennai Super Kings"}
    )

In [4]:
docs = [doc1,doc2,doc3,doc4,doc5]

In [8]:
# Embedding Setup
embedding_model = HuggingFaceEmbeddings(
    model_name = 'intfloat/e5-small',
    encode_kwargs={'normalize_embeddings':True}
)

In [9]:
# Vector Store Setup
vector_store = Chroma(
    embedding_function = embedding_model,
    persist_directory='my_chroma_db',
    collection_name='sample'
)

In [11]:
# Add Documents -> we can pass our own id as well
vector_store.add_documents(docs)

['f413bbc2-6aeb-4e09-87e1-b6ccaff004ed',
 'eddf6a56-3baf-41f3-b65d-1ebc4021d2e1',
 'dd63e793-9b42-40ad-aade-2ec03e4473ba',
 '18845c14-7dd5-492b-9cec-1f708fe20837',
 'd7b8bead-10a7-4b31-af0c-ef8166ca74ef']

In [13]:
# View documents
vector_store.get(include=['embeddings','documents','metadatas'])

{'ids': ['f413bbc2-6aeb-4e09-87e1-b6ccaff004ed',
  'eddf6a56-3baf-41f3-b65d-1ebc4021d2e1',
  'dd63e793-9b42-40ad-aade-2ec03e4473ba',
  '18845c14-7dd5-492b-9cec-1f708fe20837',
  'd7b8bead-10a7-4b31-af0c-ef8166ca74ef'],
 'embeddings': array([[ 0.01901846, -0.01498251, -0.08518275, ...,  0.01487343,
         -0.00130043,  0.08019212],
        [ 0.01963637, -0.03145466, -0.0700325 , ...,  0.00115769,
          0.00951147,  0.06269164],
        [ 0.01976116, -0.03853019, -0.12037988, ..., -0.01708871,
          0.02141129,  0.07051493],
        [-0.00270025, -0.02594804, -0.05797949, ..., -0.00857932,
          0.00691617,  0.02974599],
        [ 0.00416735, -0.04619416, -0.06100054, ...,  0.01147635,
          0.00969743,  0.07159948]]),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.',
  "Rohit Sharma is the most successful ca

In [15]:
# Search Documents
vector_store.similarity_search(
    query='Who among these are a bowler?',
    k = 2
)

[Document(metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
 Document(metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.')]

In [16]:
# Search with similarity Score
vector_store.similarity_search_with_score(
    query = "Who among these are bowler?",
    k=1
)

[(Document(metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
  0.34315598011016846)]

In [19]:
# Meta-data Filtering
vector_store.similarity_search_with_score(
    query="",
    filter = {'team':"Chennai Super Kings"}
)

[(Document(metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.'),
  0.4943499267101288),
 (Document(metadata={'team': 'Chennai Super Kings'}, page_content='MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.'),
  0.5313642024993896)]

In [20]:
# Update documents
updated_doc1 = Document(
    page_content="Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple centuries in a single season. Despite RCB not winning an IPL title under his captaincy, Kohli's passion and fitness set a benchmark for the league. His ability to chase targets and anchor innings has made him one of the most dependable players in T20 cricket.",
    metadata={"team":"Royal Challengers Bangalore"}
)
vector_store.update_document(document_id='f413bbc2-6aeb-4e09-87e1-b6ccaff004ed',document=updated_doc1)

In [21]:
vector_store.get(include=['embeddings','documents','metadatas'])

{'ids': ['f413bbc2-6aeb-4e09-87e1-b6ccaff004ed',
  'eddf6a56-3baf-41f3-b65d-1ebc4021d2e1',
  'dd63e793-9b42-40ad-aade-2ec03e4473ba',
  '18845c14-7dd5-492b-9cec-1f708fe20837',
  'd7b8bead-10a7-4b31-af0c-ef8166ca74ef'],
 'embeddings': array([[ 0.00705105, -0.03696831, -0.07145128, ..., -0.00077237,
         -0.00999008,  0.03116232],
        [ 0.01963637, -0.03145466, -0.0700325 , ...,  0.00115769,
          0.00951147,  0.06269164],
        [ 0.01976116, -0.03853019, -0.12037988, ..., -0.01708871,
          0.02141129,  0.07051493],
        [-0.00270025, -0.02594804, -0.05797949, ..., -0.00857932,
          0.00691617,  0.02974599],
        [ 0.00416735, -0.04619416, -0.06100054, ...,  0.01147635,
          0.00969743,  0.07159948]]),
 'documents': ["Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple centuries in a sin

In [22]:
# Delete Documents
vector_store.delete(ids=['dd63e793-9b42-40ad-aade-2ec03e4473ba'])

In [23]:
vector_store.get(include=['embeddings','documents','metadatas'])

{'ids': ['f413bbc2-6aeb-4e09-87e1-b6ccaff004ed',
  'eddf6a56-3baf-41f3-b65d-1ebc4021d2e1',
  '18845c14-7dd5-492b-9cec-1f708fe20837',
  'd7b8bead-10a7-4b31-af0c-ef8166ca74ef'],
 'embeddings': array([[ 0.00705105, -0.03696831, -0.07145128, ..., -0.00077237,
         -0.00999008,  0.03116232],
        [ 0.01963637, -0.03145466, -0.0700325 , ...,  0.00115769,
          0.00951147,  0.06269164],
        [-0.00270025, -0.02594804, -0.05797949, ..., -0.00857932,
          0.00691617,  0.02974599],
        [ 0.00416735, -0.04619416, -0.06100054, ...,  0.01147635,
          0.00969743,  0.07159948]]),
 'documents': ["Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple centuries in a single season. Despite RCB not winning an IPL title under his captaincy, Kohli's passion and fitness set a benchmark for the league. His ability to